In [1]:
def timer(func):
    import time
    def wrapper(*args, **kwargs):
        start = time.time()
        func(*args, **kwargs)
        end = time.time()
        print('Time : {:.10e}'.format(end - start))
    return wrapper

# 상하좌우

In [2]:
input1 = '''5
R R R U D D'''

@timer
def solution(input1):
    N = int(input1.split('\n')[0])
    array = input1.split('\n')[1:][0].split()

    loc = [1, 1]
    for cmd in array:
        if cmd == 'L' and loc[1] > 1:
            loc[1] -= 1
        elif cmd == 'R' and loc[1] < N:
            loc[1] += 1
        elif cmd == 'U' and loc[0] > 1:
            loc[0] -= 1
        elif cmd == 'D' and loc[0] < N:
            loc[0] += 1

    print(' '.join(list(map(str, loc))))
    
solution(input1)

3 4
Time : 0.0000000000e+00


In [3]:
@timer
def solution(input1):
    N = int(input1.split('\n')[0])
    plans = input1.split('\n')[1:][0].split()

    x, y = 1, 1
    dx = [0, 0, -1, 1]
    dy = [-1, 1, 0, 0]
    move_types = ['L', 'R', 'U', 'D']

    for plan in plans:
        for i in range(len(move_types)):
            if plan == move_types[i]:
                nx = x + dx[i]
                ny = y + dy[i]
        if nx < 1 or ny < 1 or nx > N or ny > N:
            continue

        x, y = nx, ny

    print(x, y)
    
solution(input1)

3 4
Time : 0.0000000000e+00


# 시각

In [18]:
input1 = '''5'''

@timer
def solution(input1):
    N = int(input1)
    cnt = 0
    s = 0
    m = 0
    h = 0
    while h <= N:
        if '3' in str(h) or '3' in str(m) or '3' in str(s):
           cnt += 1 
        s += 1
        if s == 60:
            s = 0
            m += 1
        if m == 60:
            m = 0
            h += 1

    print(cnt)

solution(input1)

11475
Time : 1.1959791183e-02


In [19]:
@timer
def solution(input1):
    N = int(input1)
    cnt = 0

    for i in range(60):
        for j in range(60):
            for k in range(N + 1):
                time = str(k) + str(j) + str(i)
                if '3' in time:
                    cnt += 1
    print(cnt)
    
solution(input1)

11475
Time : 1.5005350113e-02


# 왕실의 나이트

In [103]:
input1 = 'a1' # 2
input2 = 'c2' # 6

@timer
def solution(input1):
    import copy
    
    src = list(input1)
    
    # index로 숫자 매핑 
    al = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    num = list(map(str, [1, 2, 3, 4, 5, 6, 7, 8]))
    assert src[0] in al and src[1] in num, 'invalid location info'
    
    src[0] = str(num[al.index(src[0])])
    src = list(map(int, src))
    
#     # ord 함수는 알파벳의 유니코드를 반환
#     src[0] = ord(src[0]) - ord('a') + 1
#     src[1] = int(src[1])

    cnt = 0
    # (+2, +1), (+2, -1), (-2, +1), (-2, -1), (+1, +2), (-1, +2), (+1, -2), (-1, -2)
    # x, y좌표가 각각 +2와 -2, +1와 -1로 구성 , x, y 좌표 자리를 바꿀 시 모든 경우의 수
    for x in [2, -2]:
        for y in [1, -1]:
            for i in [True, False]:
            # copy하지 않을 경우 src가 수정되어 버림
                tmp = copy.copy(src)
                tmp[int(i)] += x
                tmp[int(not i)] += y
            # 체스판 위를 나가지 않는 경우만 append
                if 1 <= tmp[0] <= 8 and 1 <= tmp[1] <= 8:
                    cnt += 1
                    
    print(cnt)
    
solution(input1)

solution(input2)

2
Time : 0.0000000000e+00
6
Time : 0.0000000000e+00


# 게임 개발
- 순환하는 수열의 경우 나머지를 이요하면 편리
- 방향 이동 문제의 경우, 이동 경우의 수를 리스트로 관리하면 편리

In [171]:
input1 = '4 4'
input2 = '1 1 0'
input3 = '''1 1 1 1
1 0 0 1
1 1 0 1
1 1 1 1'''

input4 = '3 3'
input5 = '1 1 0'
input6 = '''1 1 0
1 1 1
1 0 0
1 1 0'''

In [170]:
@timer
def solution(input1, input2, input3):
    N, M = list(map(int, input1.split()))
    loc = list(map(int, input2.split()[:2]))
    # 0 = 위, 1 = 오른쪽, 2 = 아래, 3 = 왼쪽
    direction = int(input2.split()[2])
    moves = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    field = [list(map(int, m.split())) for m in input3.split('\n')]
    
    # 현재 위치를 지나간 곳으로 체크
    field[loc[0]][loc[1]] = -1
    cnt = 1

    # 현재 위치 기준 사방을 확인하는 함수 정의, 모두 가보지 않은 곳이면 False 반환
    def check_around(loc):
        down = field[loc[0] - 1][loc[1]] != 0
        up = field[loc[0] + 1][loc[1]] != 0
        left = field[loc[0]][loc[1] - 1] != 0 
        right = field[loc[0]][loc[1] + 1] != 0
        return False if down + up + left + right == 4 else True

    while True:
        # 1단계 : 왼쪽 회전 후 방향에 따른 이동거리 정의
        direction = (direction - 1) % 4
        dy = moves[direction][0]
        dx = moves[direction][1]
        new_loc = (loc[0] + dy, loc[1] + dx)
        # 2단계 : 왼쪽이 가보지 않은 곳일 경우 위치를 이동, 카운팅
        if field[new_loc[0]][new_loc[1]] == 0:
            field[new_loc[0]][new_loc[1]] = -1
            cnt += 1
            loc = new_loc
            continue
        # 3단계 : 사방이 다 바다 혹은 가본 곳일 경우 뒤로 이동
        if not check_around(loc):
            dy = -(moves[direction][0])
            dx = -(moves[direction][1])
            new_loc = (loc[0] + dy, loc[1] + dx)
            # 뒤가 바다일 경우 종료
            if field[new_loc[0]][new_loc[1]] == 1:
                break
            loc = new_loc
            continue
            
    print(cnt)

solution(input1, input2, input3)

3
Time : 0.0000000000e+00


In [200]:
@timer
def solution(input1, input2, input3):
    N, M = list(map(int, input1.split()))
    x, y, direction = list(map(int, input2.split()))

    dx = [-1, 0, 1, 0]
    dy = [0, 1, 0, -1]

    path = [[0] * M for _ in range(N)]
    path[x][y] = 1
    field = [list(map(int, row.split())) for row in input3.split('\n')]

    cnt = 1
    n_turn = 0
    while True:
        direction = (direction - 1) % 4
        nx = x + dx[direction]
        ny = y + dy[direction]
        if field[nx][ny] == 0 and path[nx][ny] ==0:
            path[nx][ny] = 1
            x, y = nx, ny
            cnt += 1
            n_turn = 0
            continue
        else:
            n_turn += 1

        if n_turn == 4:
            nx = x - dx[direction]
            ny = y - dy[direction]
            if field[nx][ny] == 1:
                break
            x = nx
            y = ny
            n_turn = 0

    print(cnt)        
    
solution(input1, input2, input3)

3
Time : 0.0000000000e+00


# 아기 상어
- N×N 크기의 공간에 물고기 M마리와 아기 상어 1마리가 있다. 공간은 1×1 크기의 정사각형 칸으로 나누어져 있다. 한 칸에는 물고기가 최대 1마리 존재한다.

아기 상어와 물고기는 모두 크기를 가지고 있고, 이 크기는 자연수이다. 가장 처음에 아기 상어의 크기는 2이고, 아기 상어는 1초에 상하좌우로 인접한 한 칸씩 이동한다.

아기 상어는 자신의 크기보다 큰 물고기가 있는 칸은 지나갈 수 없고, 나머지 칸은 모두 지나갈 수 있다. 아기 상어는 자신의 크기보다 작은 물고기만 먹을 수 있다. 따라서, 크기가 같은 물고기는 먹을 수 없지만, 그 물고기가 있는 칸은 지나갈 수 있다.

아기 상어가 어디로 이동할지 결정하는 방법은 아래와 같다.

더 이상 먹을 수 있는 물고기가 공간에 없다면 아기 상어는 엄마 상어에게 도움을 요청한다.
먹을 수 있는 물고기가 1마리라면, 그 물고기를 먹으러 간다.
먹을 수 있는 물고기가 1마리보다 많다면, 거리가 가장 가까운 물고기를 먹으러 간다.
거리는 아기 상어가 있는 칸에서 물고기가 있는 칸으로 이동할 때, 지나야하는 칸의 개수의 최솟값이다.
거리가 가까운 물고기가 많다면, 가장 위에 있는 물고기, 그러한 물고기가 여러마리라면, 가장 왼쪽에 있는 물고기를 먹는다.
아기 상어의 이동은 1초 걸리고, 물고기를 먹는데 걸리는 시간은 없다고 가정한다. 즉, 아기 상어가 먹을 수 있는 물고기가 있는 칸으로 이동했다면, 이동과 동시에 물고기를 먹는다. 물고기를 먹으면, 그 칸은 빈 칸이 된다.

아기 상어는 자신의 크기와 같은 수의 물고기를 먹을 때 마다 크기가 1 증가한다. 예를 들어, 크기가 2인 아기 상어는 물고기를 2마리 먹으면 크기가 3이 된다.

공간의 상태가 주어졌을 때, 아기 상어가 몇 초 동안 엄마 상어에게 도움을 요청하지 않고 물고기를 잡아먹을 수 있는지 구하는 프로그램을 작성하시오.
- 첫째 줄에 공간의 크기 N(2 ≤ N ≤ 20)이 주어진다.

둘째 줄부터 N개의 줄에 공간의 상태가 주어진다. 공간의 상태는 0, 1, 2, 3, 4, 5, 6, 9로 이루어져 있고, 아래와 같은 의미를 가진다.

0: 빈 칸
1, 2, 3, 4, 5, 6: 칸에 있는 물고기의 크기
9: 아기 상어의 위치
아기 상어는 공간에 한 마리 있다.

In [2]:
input1 = '''3
0 0 0
0 0 0
0 9 0''' # 0
input2 = '''3
0 0 1
0 0 0
0 9 0'''# 3
input3 = '''4
4 3 2 1
0 0 0 0
0 0 9 0
1 2 3 4''' #14
input4 = '''6
5 4 3 2 3 4
4 3 2 3 4 5
3 2 9 5 6 6
2 1 2 3 4 5
3 2 1 6 5 4
6 6 6 6 6 6''' #60
input5 = '''6
6 0 6 0 6 1
0 0 0 0 0 2
2 3 4 5 6 6
0 0 0 0 0 2
0 2 0 0 0 0
3 9 3 0 0 1''' #48
input6 = '''6
1 1 1 1 1 1
2 2 6 2 2 3
2 2 5 2 2 3
2 2 2 4 6 3
0 0 0 0 0 6
0 0 0 0 0 9''' #39

In [3]:
from collections import deque

N = int(input2.split('\n')[0])
graph = [list(map(int, row.split())) for row in input2.split('\n')[1:]]
visited = [[0] * N for _ in range(N)]

size = 2
feed = 0

# bfs를 사용하여 한칸씩 확장해가면서 가장 먼저 만나는 물고기를 먹는다.

# 상 하 좌 우
dx = [-1, 1, 0, 0]
dy = [0, 0, -1, 1]

def bfs(x, y):
    global feed
    visited[x][y] = 1
    dist = 0
    
    queue = deque()
    queue.append((x, y))
    
    while deque:
        print(np.array(visited))
        x, y = queue.popleft()
        for i in range(4):
            nx = x + dx[i]
            ny = y + dy[i]
            if nx < 0 or ny < 0 or nx >= N or ny >= N:
                continue
            if graph[nx][ny] == 0 and visited[nx][ny] == 0:
                visited[nx][ny] = visited[x][y] + 1
                queue.append((nx, ny))
                dist += 1
            if graph[nx][ny] < size and graph[nx][ny] != 0:
                feed += 1
                return dist
bfs(2, 1)                

[[0 0 0]
 [0 0 0]
 [0 1 0]]
[[0 0 0]
 [0 2 0]
 [2 1 2]]
[[0 3 0]
 [3 2 3]
 [2 1 2]]
[[0 3 0]
 [3 2 3]
 [2 1 2]]
[[0 3 0]
 [3 2 3]
 [2 1 2]]


7

In [5]:
from collections import deque

N = int(input2.split('\n')[0])
graph = [list(map(int, row.split())) for row in input2.split('\n')[1:]]
visited = [[0] * N for _ in range(N)]

size = 2
feed = 0

# dfs를 사용하여 재귀적으로 함수를 관리하여 최단거리를 찾음
dist = 0
def dfs(x, y):
    global dist
    dist += 1
    visited[x][y] = 1
    if graph[x][y] < size and graph[x][y] != 0:
        return dist
    if x < 0 or y < 0 or x >= N or y >= N:
        pass
    if graph[x][y] <= size and visited[x][y] == 0:
        graph[x][y] = 987654321
        dfs(x + 1, y)
        dfs(x - 1, y)
        dfs(x, y + 1)
        dfs(x, y - 1)
        return x, y
        
        
dfs(2, 1)

In [331]:
def solution(input2):
    N = int(input2.split('\n')[0])
    field = [list(map(int, row.split())) for row in input2.split('\n')[1:]]
    size = 2
    feed = 0
    total_dist = 0
    
    for n in range(N):
        if 9 in field[n]:
            x, y = n, field[n].index(9)
            
    while True:
        min_dist = 99999

        # 거리 구하기
        for i in range(N):
            for j in range(N):
                target = field[i][j]
                # 물고기가 존재하는 좌표에 대해서 좌표와 거리를 저장
                if target != 0 and target != 9 and target < size:
                    dist = abs(x - i) + abs(y - j)
                    # 최단거리인 경우, 먹을 수 있는 경우를 저장
                    if dist < min_dist:
                        min_dist = dist
                        nx, ny = i, j

        if min_dist == 99999:
            break
        total_dist += min_dist
        x, y = nx, ny
        field[x][y] = 0

        feed += 1
        if feed == size:
            feed = 0
            size += 1
            continue
        
    print(total_dist)

solution(input1)    
solution(input2)
solution(input3)
solution(input4)
solution(input5) # X
solution(input6) # X
# 경로상에 큰 물고기가 가로막고있는 경우를 처리해야 함

0
3
14
60
28
33


# 럭키 스트레이트

In [15]:
input1 = '''123402'''
input2 = '''7755'''

def solution(input1):
    array = list(map(int, input1))
    mid = len(array) // 2
    print('LUCKY' if sum(array[:mid]) == sum(array[mid:]) else 'READY')

solution(input1)
solution(input2)

LUCKY
READY


# 문자열 재정렬

In [31]:
input1 = '''K1KA5CB7'''
input2 = '''AJKDLSI412K4JSJ9D'''

def solution(input1):
    array = list(map(str, input1))
    str_array = []
    int_array = []

    for i in array:
        try:
            int_array.append(int(i))
        except:
            str_array.append((ord(i) - ord('A') + 1, i))

    result = []
    str_array.sort()
    for _, i in str_array:
        result.append(i)
    result.append(str(sum(int_array)))
    print(''.join(result))
    
solution(input1)
solution(input2)

ABCKK13
ADDIJJJKKLSS20


# 문자열 압축

In [1]:
input1 = '''aabbaccc'''
input2 = '''ababcdcdababcdcd'''
input3 = '''abcabcdede'''
input4 = '''abcabcabcabcdededededede'''
input5 = '''xababcdcdababcdcd'''
input6 = '''abcdefgabcdefg'''


def solution(input1):
    s = input1
    length = len(s)
    compressed = ''
    cnt = 1
    min_cnt = 987654321
    
    for n in range(1, length // 2 + 1):
        ngrams = []
        for i in range(0, length, n):
            if s[i:i + n] != '':
                ngrams.append(s[i:i + n])
        for i in range(len(ngrams) - 1):
            if ngrams[i] == ngrams[i + 1]:
                cnt += 1
            else:
                if cnt > 1:
                    compressed += str(cnt) + ngrams[i]
                    cnt = 1
                else:
                    compressed += ngrams[i]

        if cnt > 1:
            compressed += str(cnt) + ngrams[-1]
            cnt = 1
        else:
            compressed += ngrams[-1]

        min_cnt = min(min_cnt, len(compressed))
        compressed = ''

    print(min_cnt)
    
solution(input1)
solution(input2)
solution(input3)
solution(input4)
solution(input5)
solution(input6)

7
9
8
14
17
8


# 자물쇠와 열쇠

In [39]:
key = [[0, 0, 0], [1, 0, 0], [0, 1, 1]]
lock = [[1, 1, 1], [1, 1, 0], [1, 0, 1]]

def solution(key, lock):
    import copy

    n = len(lock)
    m = len(key)

    # 암기
    def rotate_matrix(matrix):
        n = len(matrix)
        rotated = [[0] * n for _ in range(n)]
        for i in range(n):
            for j in range(n):
                rotated[j][n - i -1] = matrix[i][j]
        return rotated

    # new_lock 초기화
    new_lock = [[0] * (2 * m + n - 2) for _ in range(2 * m + n - 2)] # 2(m-1) + n = 2m + n - 2
    for i in range(n):
        for j in range(n):
            new_lock[i + (m - 1)][j + (m - 1)] = lock[i][j]

    k = len(new_lock)

    def check(tmp, key, i, j):
        for a in range(m):
            for b in range(m):
                tmp[i + a][j + b] += key[a][b]
                
        for a in range(n):
            for b in range(n):
                if tmp[a + (m - 1)][b + (m - 1)] != 1:
                    return False
        return True

    for i in range(k - m + 1):
        for j in range(k - m + 1):
            for _ in range(4):
                key = rotate_matrix(key)
                tmp = copy.deepcopy(new_lock) # 핵심! 이중리스트 구조를 복사하기 위해서는 deepcopy를 사용 
                if check(tmp, key, i, j):
                    return True
    return False

solution(key, lock)

True

# 뱀

In [66]:
input1 = '''6
3
3 4
2 5
5 3
3
3 D
15 L
17 D''' # 9

input2 = '''10
4
1 2
1 3
1 4
1 5
4
8 D
10 D
11 D
13 L'''

input3 = '''10
5
1 5
1 3
1 2
1 6
1 7
4
8 D
10 D
11 D
13 L'''

def solution(input1):
    import heapq
    from collections import deque
    
    # 입력 초기화
    n = int(input1.split('\n')[0])
    k = int(input1.split('\n')[1])
    l = int(input1.split('\n')[2 + k])

    graph = [[0] * n for _ in range(n)]
    for row, col in [row.split() for row in input1.split('\n')[2:2 + k]]:
        graph[int(row) - 1][int(col) - 1] = 1
    
    moves = []
    for x, c in [row.split() for row in input1.split('\n')[3 + k:]]:
        moves.append((int(x), c))

    # 이동정보: 상우하좌
    directions = [0, 1, 2, 3]
    dx = [-1, 0, 1, 0]
    dy = [0, 1, 0, -1]

    # 초기정보
    d = 1
    x, y = 0, 0
    time = 0
    length = 0

    q = deque()

    while True:
        time += 1
        # 꼬리 정보는 큐로 관리
        if time > 1:
            for i in range(len(q)):
                res, prev_loc = q.popleft()
                if res == 0:
                    graph[prev_loc[0]][prev_loc[1]] = 0
                else:
                    q.append((res - 1, prev_loc))

        nx = x + dx[d]
        ny = y + dy[d]
        
        # 벽에 충돌 시 종료
        if nx < 0 or ny < 0 or nx >= n or ny >= n:
            break
        # 꼬리에 충돌 시 종료
        if graph[nx][ny] == -1:
            break
        # 사과를 먹을 시 길이 증가
        if graph[nx][ny] == 1:
            length += 1
        
        # 이전 위치(꼬리 위치)에 -1
        graph[x][y] = -1
        q.append((length, (x, y)))
        # 꼬리는 당시 뱀의 길이만큼 유지됨
        # 꼬리 정보는 최소 힙으로 관리
        if len(moves) > 0:
            reserved, move = heapq.heappop(moves)
            if reserved == time:
                if move == 'L':
                    d = (d - 1) % 4 
                else: 
                    d = (d + 1) % 4
            else:
                heapq.heappush(moves, (reserved, move))
        
        x, y = nx, ny
    print(time)

solution(input1)
solution(input2)
solution(input3)

9
21
13


# 기둥과 보 설치

In [96]:
n1 = 5
build_frame1 = [[1, 0, 0, 1], [1, 1, 1, 1], [2, 1, 0, 1], [2, 2, 1, 1],
               [5, 0, 0, 1], [5, 1, 0, 1], [4, 2, 1, 1], [3, 2, 1, 1]]

n2 = 5
build_frame2 = [[0, 0,0, 1], [2, 0, 0, 1], [4, 0, 0, 1], [0, 1, 1, 1], [1, 1, 1, 1], [2, 1, 1, 1], 
               [3, 1, 1, 1], [2, 0, 0, 0], [1, 1, 1, 0], [2, 2, 0, 1]]

In [69]:
def solution(n, build_frame):
    graph = [[-1] * (n + 1) for _ in range(n + 1)]
    result = []
    
    def make_column(x, y, a):
        if x == n or graph[x][y] == 1 or graph[x][y] == 0:
            graph[x][y] = 0
            graph[x - 1][y] = 0
            result.append((y, n - x, a))

    def delete_column(x, y, a):
        if x < n:
            if graph[x - 1][y] != -1:
                return
        graph[x][y] = -1
        result.remove((y, n - x, a))

    def make_ceil(x, y, a):
        if graph[x][y] == 0 or graph[x][y] == 1:
            graph[x][y] = 1
            graph[x][y + 1] = 1
            result.append((y, n - x, b))
            return
        if y < n:
            if graph[x][y + 1] == 0:
                graph[x][y] = 1
                graph[x][y + 1] = 1
                result.append((y, n - x, b))

    def delete_ceil(x, y, a):
        if x > 1:
            if graph[x - 1][y] == 1:
                return
        if y > 2:
            if graph[x][y - 1] == 1 and graph[x][y - 2] == 1 and graph[x + 1][y - 1] != 0:
                return
        if y < n - 1:
            if graph[x][y + 1] == 1 and graph[x][y + 2] == 1 and graph[x + 1][y + 1] != 0:
                return
        graph[x][y] = -1
        result.remove((y, n - x, b))

    for x, y, a, b in build_frame:
        x, y = n - y, x
        if a == 0:
            if b == 1:
                make_column(x, y, a)
            if b == 0:
                delete_column(x, y, a)
        if a == 1:
            if b == 1:
                make_ceil(x, y, a)
            if b == 0:
                delete_ceil(x, y, a)

    for i in range(2, -1, -1):
        result = sorted(result, key=itemgetter(i))
        
    print(result)
    
solution(n1, build_frame1)
solution(n2, build_frame2)

[(1, 0, 0), (1, 1, 1), (2, 1, 0), (2, 2, 1), (3, 2, 1), (4, 2, 1), (5, 0, 0), (5, 1, 0)]
[(0, 0, 0), (0, 1, 1), (1, 1, 1), (2, 1, 1), (3, 1, 1), (4, 0, 0)]


In [98]:
def possible(answer):
    for x, y, a in answer:
        if a == 0:
            if y == 0 or (x, y - 1, 0) in answer or (x, y, 1) in answer or (x - 1, y, 1) in answer:
                continue
            return False
        if a == 1:
            if ((x - 1, y, 1) in answer and (x + 1, y, 1) in answer) \
            or (x, y - 1, 0) in answer or (x + 1, y - 1, 0) in answer:
                continue
            return False
    return True
    
def solution(n, build_frame):
    from operator import itemgetter
    
    answer = []
    for x, y, a, b in build_frame:
        if b == 0:
            answer.remove((x, y, a))
            if not possible(answer):
                answer.append((x, y, a))
        if b == 1:
            answer.append((x, y, a))
            if not possible(answer):
                answer.remove((x, y, a))
    for i in range(2, -1, -1):
        answer = sorted(answer, key=itemgetter(i))
    return answer

print(solution(n1, build_frame1))
print(solution(n2, build_frame2))

[(1, 0, 0), (1, 1, 1), (2, 1, 0), (2, 2, 1), (3, 2, 1), (4, 2, 1), (5, 0, 0), (5, 1, 0)]
[(0, 0, 0), (0, 1, 1), (1, 1, 1), (2, 1, 1), (3, 1, 1), (4, 0, 0)]


# 치킨 배달

In [137]:
input1 = '''5 3
0 0 1 0 0
0 0 2 0 1
0 1 2 0 0
0 0 1 0 0
0 0 0 0 2'''

input2 = '''5 2
0 2 0 1 0
1 0 1 0 0
0 0 0 0 0
2 0 0 1 1
2 2 0 1 2'''

input3 = '''5 1
1 2 0 0 0
1 2 0 0 0
1 2 0 0 0
1 2 0 0 0
1 2 0 0 0'''
# 집과의 거리 합이 가장 먼 치킨집을 뻄

def get__min_dist(x, y, chicken):
        min_dist = 987654321
        for i, j in chicken:
            dist = abs(i - x) + abs(j - y)
            if min_dist > dist:
                min_dist = dist
        return min_dist

def get_total_dist(x, y, home):
    total_dist = 0
    for i, j in home:
        dist = abs(i - x) + abs(j - y)
        total_dist += dist
    return total_dist

def solution(input1):
    import math

    n, m = list(map(int, input1.split('\n')[0].split()))
    graph = [list(map(int, row.split())) for row in input1.split('\n')[1:]]
    
    # 집 위치 저장
    home = []
    for i in range(n):
        for j in range(n):
            if graph[i][j] == 1:
                home.append((i, j))

    # 치킨집의 모든 집과의 거리합을 저장
    # 가장 거리합이 낮은 치킨집들을 폐업시키지 않음
    result = []
    for i in range(n):
        for j in range(n):
            if graph[i][j] == 2:
                dist = get_total_dist(i, j, home)
                result.append((dist, i, j))
    result.sort()

    # 남은 치킨집 위치를 저장
    chicken = []
    answer = 0
    for _, x, y in result[:m]:
        chicken.append((x, y))

    # 최소합 구하기
    for i in range(n):
        for j in range(n):
            if graph[i][j] == 1:
                dist = get_min_dist(i, j, chicken)
                answer += dist
    print(answer)
    
solution(input1)
solution(input2)
solution(input3)

5
10
11


In [151]:
def solution(input2):
    from itertools import combinations

    n, m = list(map(int, input2.split('\n')[0].split()))
    graph = [list(map(int, row.split())) for row in input2.split('\n')[1:]]

    home = []
    chicken = []
    for i in range(n):
        for j in range(n):
            if graph[i][j] == 2:
                chicken.append((i, j))
            elif graph[i][j] == 1:
                home.append((i, j))

    def get_total_dist(c):
        total_d = 0
        for a, b in home:
            min_d = 987654321
            for x, y in c:
                d = abs(x - a) + abs(y - b)
                if min_d > d:
                    min_d = d
            total_d += min_d
        return total_d

    cand = list(combinations(chicken, m))
    min_dist = 987654321
    for c in cand:
        dist = get_total_dist(c)
        if min_dist > dist:
            min_dist = dist
    print(min_dist)
    
solution(input1)
solution(input2)
solution(input3)

5
10
11


# 외벽 점검

레스토랑을 운영하고 있는 "스카피"는 레스토랑 내부가 너무 낡아 친구들과 함께 직접 리모델링 하기로 했습니다. 레스토랑이 있는 곳은 스노우타운으로 매우 추운 지역이어서 내부 공사를 하는 도중에 주기적으로 외벽의 상태를 점검해야 할 필요가 있습니다.

레스토랑의 구조는 완전히 동그란 모양이고 외벽의 총 둘레는 n미터이며, 외벽의 몇몇 지점은 추위가 심할 경우 손상될 수도 있는 취약한 지점들이 있습니다. 따라서 내부 공사 도중에도 외벽의 취약 지점들이 손상되지 않았는 지, 주기적으로 친구들을 보내서 점검을 하기로 했습니다. 다만, 빠른 공사 진행을 위해 점검 시간을 1시간으로 제한했습니다. 친구들이 1시간 동안 이동할 수 있는 거리는 제각각이기 때문에, 최소한의 친구들을 투입해 취약 지점을 점검하고 나머지 친구들은 내부 공사를 돕도록 하려고 합니다. 편의 상 레스토랑의 정북 방향 지점을 0으로 나타내며, 취약 지점의 위치는 정북 방향 지점으로부터 시계 방향으로 떨어진 거리로 나타냅니다. 또, 친구들은 출발 지점부터 시계, 혹은 반시계 방향으로 외벽을 따라서만 이동합니다.

외벽의 길이 n, 취약 지점의 위치가 담긴 배열 weak, 각 친구가 1시간 동안 이동할 수 있는 거리가 담긴 배열 dist가 매개변수로 주어질 때, 취약 지점을 점검하기 위해 보내야 하는 친구 수의 최소값을 return 하도록 solution 함수를 완성해주세요.

In [62]:
def solution(n, weak, dist):
    from itertools import permutations

    length = len(weak)
    weak += [i + n for i in weak]
    # 시작 지점 : 0 ~ len(weak) - 1
    # 탐색 : 시작 지점 + len(weak)
    # 모든 시작지점에 대해서 iterate 및 결과 저장, 업데이트
    answer = len(dist) + 1

    for start in range(length): # max(length) = 15
        limit = start + length # 여기가 문제
        for line in permutations(dist, len(dist)):
            count = 1
            end = weak[start] + line[count - 1]
            
            for i in range(start, limit):
                if weak[i] > end:
                    count += 1
                    if count > len(dist):
                        break
                    end = weak[i] + line[count - 1]
            if count < answer:
                answer = count
                
    if answer > len(dist):
        answer = - 1
        
    return answer

solution(n, weak, dist)

2

# 청소년 상어

In [111]:
INPUT = '''7 6 2 3 15 6 9 8
3 1 1 8 14 7 10 1
6 1 13 6 4 3 11 4
16 1 8 7 5 2 12 2'''

INPUT = '''16 7 1 4 4 3 12 8
14 7 7 6 3 4 10 2
5 2 15 2 8 3 6 4
11 8 2 4 13 5 9 4'''

In [114]:
for i in range(4):
    row = list(map(int, input().split()))
    for j in range(4):
        array[i][j] = [row[2 * j], row[2 * j + 1] - 1]
        
array

16 7 1 4 4 3 12 8
[16, 7, 1, 4, 4, 3, 12, 8]
14 7 7 6 3 4 10 2
[14, 7, 7, 6, 3, 4, 10, 2]


KeyboardInterrupt: Interrupted by user

In [ ]:
from copy import deepcopy


array = [[None] * 4 for _ in range(4)]

for i in range(4):
    row = list(map(int, input().split()))
    for j in range(4):
        array[i][j] = [row[2 * j], row[2 * j + 1] - 1]

dx = [-1, -1, 0, 1, 1, 1, 0, -1]
dy = [0, -1, -1, -1, 0, 1, 1, 1]

def find(array, idx):
    for i in range(4):
        for j in range(4):
            if array[i][j][0] == idx:
                return (i, j)
    return None

def fish_moving(shark, array):
    sx, sy = shark
    for i in range(1, 17):
        pos = find(array, i)
        if pos:
            x, y = pos
            if (x, y) == (sx, sy):
                continue
            direction = array[x][y][1]
            for _ in range(8):
                nx, ny = x + dx[direction], y + dy[direction]
                if 0 <= nx < 4 and 0 <= ny < 4:
                    if (nx, ny) != (sx, sy):
                        array[x][y][1] = direction
                        array[x][y], array[nx][ny] = array[nx][ny], array[x][y]
                        break

                direction = (direction + 1) % 8

def possible_position(array, shark):
    possible = []
    sx, sy = shark
    direction = array[sx][sy][1]
    
    for _ in range(4):
        sx += dx[direction]
        sy += dy[direction]
        if 0 <= sx < 4 and 0 <= sy < 4:
            if array[sx][sy][0] != -1:
                possible.append((sx, sy))
                
    return possible

max_cnt = 0

def dfs(shark, array, cnt):
    global max_cnt
    
    x, y = shark
    array = deepcopy(array)

    cnt += array[x][y][0]
    array[x][y][0] = -1

    fish_moving(shark, array)
    possible = possible_position(array, shark)

    if len(possible) == 0:
        max_cnt = max(cnt, max_cnt)
        return

    for a, b in possible:
        dfs((a, b), array, cnt)

shark = 0, 0
cnt = 0
dfs(shark, array, cnt)
print(max_cnt)

In [267]:
INPUT = '''4 2 6
1 0 0 0
0 0 0 0
0 0 0 0
0 0 0 2
4 3
1 2 3 4
2 3 4 1
3 4 1 2
4 1 2 3
1 2 3 4
2 3 4 1
3 4 1 2
4 1 2 3'''

INPUT = '''5 4 4
0 0 0 0 3
0 2 0 0 0
1 0 0 0 4
0 0 0 0 0
0 0 0 0 0
4 4 3 1
2 3 1 4
4 1 2 3
3 4 2 1
4 3 1 2
2 4 3 1
2 1 3 4
3 4 1 2
4 1 2 3
4 3 2 1
1 4 3 2
1 3 2 4
3 2 1 4
3 4 1 2
3 2 4 1
1 4 2 3
1 4 2 3'''

# INPUT = '''5 4 1
# 0 0 0 0 3
# 0 2 0 0 0
# 1 0 0 0 4
# 0 0 0 0 0
# 0 0 0 0 0
# 4 4 3 1
# 2 3 1 4
# 4 1 2 3
# 3 4 2 1
# 4 3 1 2
# 2 4 3 1
# 2 1 3 4
# 3 4 1 2
# 4 1 2 3
# 4 3 2 1
# 1 4 3 2
# 1 3 2 4
# 3 2 1 4
# 3 4 1 2
# 3 2 4 1
# 1 4 2 3
# 1 4 2 3'''

# INPUT = '''5 4 10
# 0 0 0 0 3
# 0 0 0 0 0
# 1 2 0 0 0
# 0 0 0 0 4
# 0 0 0 0 0
# 4 4 3 1
# 2 3 1 4
# 4 1 2 3
# 3 4 2 1
# 4 3 1 2
# 2 4 3 1
# 2 1 3 4
# 3 4 1 2
# 4 1 2 3
# 4 3 2 1
# 1 4 3 2
# 1 3 2 4
# 3 2 1 4
# 3 4 1 2
# 3 2 4 1
# 1 4 2 3
# 1 4 2 3'''

N, M, k = list(map(int, INPUT.split('\n')[0].split()))
array = [list(map(int, row.split())) for row in INPUT.split('\n')[1 : 1 + N]]
direction = list(map(int, INPUT.split('\n')[1 + N].split()))

import copy

pref = []
for i in range(M):
    pref.append([list(map(int, row.split())) for row in INPUT.split('\n')[(2 + N) + i * 4 : (2 + N) + (i + 1) * 4]])

time = [[[0, 0] for _ in range(N)] for _ in range(N)]
for i in range(N):
    for j in range(N):
        if array[i][j] != 0:
            time[i][j][0] = array[i][j]
            time[i][j][1] = k
dx = [-1, 1, 0, 0]
dy = [0, 0, -1, 1]

def find_shark(array, idx):
    for i in range(N):
        for j in range(N):
            if array[i][j] == idx:
                return i, j
    return None

def time_pass():
    for a in range(N):
        for b in range(N):
            if time[a][b][1] > 0:
                time[a][b][1] -= 1
                if time[a][b][1] == 0:
                    time[a][b][0] = 0

def next_location(x, y):
    for j in pref[array[x][y] - 1][direction[array[x][y] - 1] - 1]:
        nx, ny = x + dx[j - 1], y + dy[j - 1]
        if 0 <= nx < N and 0 <= ny < N:
            if time[nx][ny][0] == 0:
                return (nx, ny), j
            
    for j in pref[array[x][y] - 1][direction[array[x][y] - 1] - 1]:
        nx, ny = x + dx[j - 1], y + dy[j - 1]
        if 0 <= nx < N and 0 <= ny < N:
            if time[nx][ny][0] == time[x][y][0]:
                return (nx, ny), j
            
    return None

def update(new_loc, new_direction, now):
    x, y = now
    nx, ny = new_loc
    direction[array[x][y] - 1] = new_direction
    array[nx][ny] = array[x][y]
    array[x][y] = 0
    time[nx][ny] = [array[nx][ny], k]
    
time = copy.deepcopy(time)
array = copy.deepcopy(array)
t = 0

for _ in range(1000):
    prev = []
    for i in range(M):
        now = find_shark(array, i + 1)
        if now is not None:
            x, y = now
            new_loc, new_direction = next_location(x, y)
            if new_loc is not None:
                if new_loc not in [p[0] for p in prev]:
                    prev.append((new_loc, new_direction, now))
                else:
                    direction[array[x][y] - 1] = 0
                    array[x][y] = 0
    
    time_pass()
    for x, y, z in prev:
        update(x, y, z)
    t += 1

    if sum(direction[1:]) == 0:
        print(t)
        break
else:
    print(-1)


14
[3, 0, 0, 0]


In [257]:
import copy

N, M, k = list(map(int, input().split()))
array = []
for _ in range(N):
    array.append(list(map(int, input().split())))
direction = list(map(int, input().split()))
pref = []
for i in range(M):
    tmp = []
    for _ in range(4):
        tmp.append(list(map(int, input().split())))
    pref.append(tmp)

time = [[[0, 0] for _ in range(N)] for _ in range(N)]

for i in range(N):
    for j in range(N):
        if array[i][j] != 0:
            time[i][j][0] = array[i][j]
            time[i][j][1] = k
            
dx = [-1, 1, 0, 0]
dy = [0, 0, -1, 1]

def find_shark(array, idx):
    for i in range(N):
        for j in range(N):
            if array[i][j] == idx:
                return i, j
    return None

def time_pass():
    for a in range(N):
        for b in range(N):
            if time[a][b][1] > 0:
                time[a][b][1] -= 1
                if time[a][b][1] == 0:
                    time[a][b][0] = 0

def next_location(x, y):
    for j in pref[array[x][y] - 1][direction[array[x][y] - 1] - 1]:
        nx, ny = x + dx[j - 1], y + dy[j - 1]
        if 0 <= nx < N and 0 <= ny < N:
            if time[nx][ny][0] == 0:
                return (nx, ny), j
            
    for j in pref[array[x][y] - 1][direction[array[x][y] - 1] - 1]:
        nx, ny = x + dx[j - 1], y + dy[j - 1]
        if 0 <= nx < N and 0 <= ny < N:
            if time[nx][ny][0] == time[x][y][0]:
                return (nx, ny), j
            
    return None

def update(new_loc, new_direction, now):
    x, y = now
    nx, ny = new_loc
    direction[array[x][y] - 1] = new_direction
    array[nx][ny] = array[x][y]
    array[x][y] = 0
    time[nx][ny] = [array[nx][ny], k]
    
time = copy.deepcopy(time)
array = copy.deepcopy(array)
t = 0

for _ in range(1000):
    prev = []
    for i in range(M):
        now = find_shark(array, i + 1)
        if now is not None:
            x, y = now
            new_loc, new_direction = next_location(x, y)
            if new_loc is not None:
                if new_loc not in [p[0] for p in prev]:
                    prev.append((new_loc, new_direction, now))
                else:
                    direction[array[x][y] - 1] = 0
                    array[x][y] = 0
    
    time_pass()
    for x, y, z in prev:
        update(x, y, z)
    t += 1
    
    if sum(direction[1:]) == 0:
        answer = t
        break
else:
    answer = -1
    
print(answer)

5 4 4
0 0 0 0 3
0 2 0 0 0
1 0 0 0 4
0 0 0 0 0
0 0 0 0 0
4 4 3 1
2 3 1 4
4 1 2 3
3 4 2 1
4 3 1 2
2 4 3 1
2 1 3 4
3 4 1 2
4 1 2 3
4 3 2 1
1 4 3 2
1 3 2 4
3 2 1 4
3 4 1 2
3 2 4 1
1 4 2 3
1 4 2 3
14
